# Credit Spreads vs Equity Cushion
© 2025 Marek Ozana

This notebook uses **polars-bloomberg** to study how bond spreads relate to a company’s **equity cushion**, (EV-NetDebt) / EV

We plot each issuer as a scatter point using **average z-spread per year of duration** versus equity cushion and highlight the typical “hockey-stick” pattern: spreads are fairly stable at healthy cushions, then widen rapidly when the cushion becomes thin.

Implementation detail: in Bloomberg BQL we map each bond to its issuer fundamentals via **`fundamentalTicker` (mapby=LINEAGE)**, then **group by `ticker` and rating** to aggregate bond-level spread/duration and issuer-level fundamentals into one issuer datapoint.



In [1]:
import json
from pathlib import Path

import altair as alt

alt.renderers.set_embed_options(actions=False)
chart_file = Path("docs/examples/credit") / "equity-cushion-vs-spread-hero.json"
with open(chart_file) as f:
    spec = json.load(f)
alt.Chart.from_dict(spec)

alt.LayerChart(...)

In [2]:
# get average spread (spread / duration) and equity cusion for all tickers

from polars_bloomberg import BQuery

query = """
    let(
        #dur = avg(group(duration(duration_type=MODIFIED),
                         by=[ticker, bb_composite]));
        #zspread = avg(group(spread(spread_type=Z),
                             by=[ticker, bb_composite]));
        #ev = avg(group(value(curr_entp_val(fpt='BT', fill='PREV'),
                              fundamentalTicker,
                              mapby=LINEAGE).value,
                        by=[ticker, bb_composite]));
        #ndebt = avg(group(value(net_debt(fpt='BT', fill='PREV'),
                                 fundamentalTicker,
                                 mapby=LINEAGE).value,
                           by=[ticker, bb_composite]));
        #ndebt2ebitda = avg(group(
            value(net_debt_to_ebitda(fpt='BT', fill='PREV'),
                  fundamentalTicker, mapby=LINEAGE).value,
            by=[ticker, bb_composite]));
        #eq_cushion = dropna((#ev - #ndebt) / #ev);
        #spread_per_year = dropna(#zspread / #dur);
    )
    get(#eq_cushion, #spread_per_year, #ndebt2ebitda)
    for(members(['LG30TRUU Index']))
"""

with BQuery() as bq:
    res = bq.bql(query)

df = res.combine().drop_nulls()
df.head(3)


ID,#eq_cushion,ORIG_IDS,TICKER,BB_COMPOSITE,#spread_per_year,DATE,#ndebt2ebitda
str,f64,str,str,str,f64,date,f64
"""ABEGET:B-""",0.332338,"""YJ291755 Corp""","""ABEGET""","""B-""",756.857051,2025-12-19,5.402544
"""ACALTD:BB""",0.785345,"""BM122057 Corp""","""ACALTD""","""BB""",-10837.804437,2025-12-19,1.572696
"""ACKAF:B""",0.363201,"""ZI967285 Corp""","""ACKAF""","""B""",130.281249,2025-12-19,4.563269


In [3]:
# Create Chart of average spread per year vs equity cushion
import altair as alt
import polars as pl

g_data = (
    df.filter(pl.col("#eq_cushion").is_between(0, 1))
    .filter(pl.col("#spread_per_year").is_between(0, 900))
    .filter(pl.col("#ndebt2ebitda").is_between(0, 10))
)
base = alt.Chart(g_data).encode(
    x=alt.X("#eq_cushion").axis(format="%").title("Equity Cushion"),
    y=alt.Y("#spread_per_year").title("Average Z-Spread per Year"),
)
points = base.mark_circle(stroke="gray").encode(
    color=alt.Color("#ndebt2ebitda:Q").scale(scheme="blueorange").title("Leverage"),
    tooltip=["TICKER", "BB_COMPOSITE", "#spread_per_year", "#eq_cushion"],
)
text = base.mark_text(align="center", baseline="bottom", opacity=0.5).encode(
    text=alt.Text("ID")
)
fit = base.transform_regression(
    "#eq_cushion", "#spread_per_year", method="pow"
).mark_line(size=3)

chart = (points + fit + text).properties(
    width=600, height=350, title="Average Z-Spread per Year vs Equity Cushion"
)
# Save the chart
chart.save(chart_file)
chart.save(chart_file.with_suffix(".png"), scale_factor=0.75)
